# Plate QC

Quality control & corrections of the cardiac differentiation plate

In [1]:
import ngio
from ngio.utils import fractal_fsspec_store
import pandas as pd
from aiohttp import ServerDisconnectedError

In [ ]:
plate_url = "https://path/to/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr"
token = ""
plate_store = fractal_fsspec_store(url=plate_url, fractal_token=token)

In [5]:
plate = ngio.open_ome_zarr_plate(plate_store, parallel_safe=False)

In [6]:
plate.list_image_tables(mode="all")

['FOV_ROI_table',
 'well_ROI_table',
 'nuclei_measurements_apx',
 'nuclei_ROI_table',
 'nuclei_measurements_scmultiplex',
 'nuclei_measurements_apx_aggregated']

In [7]:
plate.list_tables()

['registration_errors', 'differentiation_timepoints']

In [8]:
registration_errors_table = plate.get_table('registration_errors').dataframe
differentiation_timepoints_table = plate.get_table('differentiation_timepoints').dataframe
lookup = differentiation_timepoints_table[["well", "day"]].rename(
    columns={"day": "differentiation_day"}
)

In [9]:
differentiation_timepoints_table

,well,day,row,column
0,B03,0,B,3
1,B05,6,B,5
2,B09,5,B,9
3,B11,0,B,11
4,C04,1,C,4
5,C06,9,C,6
6,C08,2,C,8
7,C10,8,C,10
8,D05,7,D,5
9,D09,4,D,9


In [10]:
# Load the feature tables into memory as pandas DataFrames
# Loop over all acquisition 0 images, load their feature tables, and concatenate them
reference_acquisitions = plate.images_paths(acquisition=0)

In [ ]:
# Debugging: Just process one well
# reference_acquisitions = ['B/05/0']

In [11]:
counter_qc = 0
counter_registration = 0
counter_segmentation = 0
concatenated_df = pd.DataFrame()

In [13]:
def load_qced_feature_table_for_well(plate, img_path, registration_errors_table, counter_qc, counter_registration, counter_segmentation):
    img = plate.get_image(row=img_path.split('/')[0], column=img_path.split('/')[1], image_path=img_path.split('/')[2])
    feature_table = img.get_table('nuclei_measurements_apx_aggregated')
    df = feature_table.dataframe.reset_index()
    
    # Kick out FOVs that occur in the registration_errors_table
    curr_well = df["well_name"][0]
    if curr_well in registration_errors_table["well"].values:
        misregistered_fovs = registration_errors_table[registration_errors_table["well"] == curr_well]
        for fov in misregistered_fovs["FOV"]:
            n = len(df[df["ROI"] == fov])
            print(
                f"Removing {fov} from well {curr_well} due to "
                f"registration errors. This removes {n} cells."
            )
            counter_qc += n
            counter_registration += n
            df = df[df["ROI"] != fov]
    
    # Kick out missegmented cells
    n_missegmented = sum(df["segmentation_qc"] != "Regular cells")
    print(
        f"Removing missegmented cells from {curr_well}: "
        f"This removes {n_missegmented} cells."
    )
    counter_qc += n_missegmented
    counter_segmentation += n_missegmented
    df = df[df["segmentation_qc"] == "Regular cells"]
    print(f"Loaded {len(df)} cells after QC from well {curr_well}.")
    return df, counter_qc, counter_registration, counter_segmentation

In [14]:
remaining_acquisitions = reference_acquisitions.copy()

In [17]:
to_be_processed_acquisitions = remaining_acquisitions.copy()
to_be_processed_acquisitions

[]

In [16]:
for img_path in to_be_processed_acquisitions:
    try:
        df, counter_qc, counter_registration, counter_segmentation = load_qced_feature_table_for_well(plate, img_path, registration_errors_table, counter_qc, counter_registration, counter_segmentation)
        print(f"Done processing well {img_path}")
        concatenated_df = pd.concat([concatenated_df, df])
        remaining_acquisitions.remove(img_path)
        print(f"Done concatenating data for well {img_path}")
    except ServerDisconnectedError:
        print(f"ServerDisconnectedError encountered while processing well {img_path}. Retrying later.")
    

print(f"Total number of removed cells during QC: {counter_qc}")
print(
    f" - of which {counter_registration} due to registration errors, "
    f"and {counter_segmentation} due to missegmentation."
)

Removing FOV_26 from well B09 due to registration errors. This removes 1439 cells.
Removing FOV_39 from well B09 due to registration errors. This removes 602 cells.
Removing FOV_40 from well B09 due to registration errors. This removes 204 cells.
Removing FOV_47 from well B09 due to registration errors. This removes 881 cells.
Removing FOV_48 from well B09 due to registration errors. This removes 1091 cells.
Removing FOV_53 from well B09 due to registration errors. This removes 344 cells.
Removing FOV_54 from well B09 due to registration errors. This removes 870 cells.
Removing FOV_55 from well B09 due to registration errors. This removes 1274 cells.
Removing FOV_56 from well B09 due to registration errors. This removes 470 cells.
Removing FOV_61 from well B09 due to registration errors. This removes 257 cells.
Removing FOV_62 from well B09 due to registration errors. This removes 695 cells.
Removing FOV_63 from well B09 due to registration errors. This removes 835 cells.
Removing FOV_

In [18]:
print(f"Total number of removed cells during QC: {counter_qc}")
print(
    f" - of which {counter_registration} due to registration errors, "
    f"and {counter_segmentation} due to missegmentation."
)

Total number of removed cells during QC: 167691
 - of which 21949 due to registration errors, and 145742 due to missegmentation.


In [19]:
concatenated_df = concatenated_df.merge(
    lookup,
    how="left",
    left_on="well_name",
    right_on="well",
)
concatenated_df = concatenated_df.drop(columns=["well"])
concatenated_df

,label,nuclei_filtered_Morphology_area,nuclei_filtered_Morphology_centroid-0,nuclei_filtered_Morphology_centroid-1,nuclei_filtered_Morphology_well_centroid-0,nuclei_filtered_Morphology_well_centroid-1,nuclei_filtered_Morphology_bbox_area,nuclei_filtered_Morphology_bbox-0,nuclei_filtered_Morphology_bbox-1,nuclei_filtered_Morphology_bbox-2,...,is_border_internal,is_border_external,well_name,ROI,segmentation_qc,nanog_classifier,Sox17_classifier,NKX25_classifier,TroponinT_classifier,differentiation_day
0,1,3997.0,70.500122,56.327995,70.500122,56.327995,10472.0,0.0,19.0,136.0,...,0.0,1.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
1,2,4307.0,38.224518,545.921082,38.224518,545.921082,5727.0,0.0,509.0,83.0,...,0.0,1.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_positive,NKX25_negative,TroponinT_negative,5
2,4,2322.0,23.260983,817.758850,23.260983,817.758850,3213.0,0.0,786.0,51.0,...,0.0,1.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
3,5,2195.0,21.335764,875.819153,21.335764,875.819153,2703.0,0.0,849.0,51.0,...,0.0,1.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
4,6,3996.0,33.656155,949.489502,33.656155,949.489502,5688.0,0.0,909.0,79.0,...,0.0,1.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041835,74238,1562.0,2148.081299,1152.081909,19428.082031,3712.082031,2100.0,2132.0,1114.0,2160.0,...,0.0,1.0,G10,FOV_66,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6
1041836,74239,1345.0,2147.655029,1777.846802,19427.654297,6897.846680,1708.0,2132.0,1748.0,2160.0,...,0.0,1.0,G10,FOV_67,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6
1041837,74242,1233.0,2148.418457,1085.787476,19428.417969,8765.787109,1534.0,2134.0,1056.0,2160.0,...,0.0,1.0,G10,FOV_68,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6
1041838,74246,1390.0,2148.655518,431.275543,19428.656250,431.275543,1675.0,2135.0,398.0,2160.0,...,0.0,1.0,G10,FOV_65,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6


In [20]:
from ngio.tables import FeatureTable
aggregated_ngio_table = FeatureTable(
    table_data=concatenated_df,
    reference_label="nuclei_filtered",
)

local_plate_url = "/Users/joel/Documents/TestDatasets/23well_benchmark/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr"
local_plate = ngio.open_ome_zarr_plate(local_plate_url)
local_plate.add_table(
    name="nuclei_measurements_initial_qced",
    table=aggregated_ngio_table,
    backend="parquet",
    overwrite=True,
)


In [21]:
print(list(concatenated_df.columns))

['label', 'nuclei_filtered_Morphology_area', 'nuclei_filtered_Morphology_centroid-0', 'nuclei_filtered_Morphology_centroid-1', 'nuclei_filtered_Morphology_well_centroid-0', 'nuclei_filtered_Morphology_well_centroid-1', 'nuclei_filtered_Morphology_bbox_area', 'nuclei_filtered_Morphology_bbox-0', 'nuclei_filtered_Morphology_bbox-1', 'nuclei_filtered_Morphology_bbox-2', 'nuclei_filtered_Morphology_bbox-3', 'nuclei_filtered_Morphology_convex_area', 'nuclei_filtered_Morphology_eccentricity', 'nuclei_filtered_Morphology_equivalent_diameter', 'nuclei_filtered_Morphology_euler_number', 'nuclei_filtered_Morphology_extent', 'nuclei_filtered_Morphology_filled_area', 'nuclei_filtered_Morphology_major_axis_length', 'nuclei_filtered_Morphology_minor_axis_length', 'nuclei_filtered_Morphology_orientation', 'nuclei_filtered_Morphology_perimeter', 'nuclei_filtered_Morphology_solidity', 'nuclei_filtered_Morphology_roundness', 'nuclei_filtered_Intensity_max_intensity_DAPI_1', 'nuclei_filtered_Intensity_me

In [22]:
# Additional QC filters
concatenated_df = concatenated_df[concatenated_df["nuclei_filtered_Morphology_solidity"] > 0.8]

In [23]:
cols_to_drop = [
    "nuclei_filtered_Population_mean_distance_nn_100",
    "nuclei_filtered_Population_mean_distance_nn_50",
]

concatenated_df.drop(columns=cols_to_drop, inplace=True)
concatenated_df.dropna(
    subset=["nuclei_filtered_Population_mean_distance_nn_25"],
    inplace=True,
)

/var/folders/sk/fl32m0g97kv2xlqz9f0p5bbm0000gr/T/ipykernel_71029/821250467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df.drop(columns=cols_to_drop, inplace=True)
/var/folders/sk/fl32m0g97kv2xlqz9f0p5bbm0000gr/T/ipykernel_71029/821250467.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df.dropna(


In [24]:
margin = 50

to_discard = (
    (concatenated_df["nuclei_filtered_Morphology_bbox-0"] < margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-1"] < margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-2"] > 2160 - margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-3"] > 2560 - margin)
)
concatenated_df = concatenated_df.loc[~to_discard]

In [25]:
# Columns to drop because they are static
# static_cols = concatenated_df.nunique(dropna=True) <= 1
# static_columns = static_cols[static_cols].index.tolist()
# print(static_columns)

cols_to_drop = [
    'nuclei_filtered_Morphology_euler_number',
    'is_border_internal',
    'is_border_external',
    'segmentation_qc'
]

concatenated_df.drop(columns=cols_to_drop, inplace=True)

In [26]:
# Save QCed table to disk
from ngio.tables import FeatureTable
qc_ngio_table = FeatureTable(
    table_data=concatenated_df,
    reference_label="nuclei_filtered",
)

local_plate_url = "/Users/joel/Documents/TestDatasets/23well_benchmark/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr"
local_plate = ngio.open_ome_zarr_plate(local_plate_url)
local_plate.add_table(
    name="nuclei_measurements_qced",
    table=qc_ngio_table,
    backend="parquet",
    overwrite=True,
)

In [27]:
CLASSIFIERS = {
    "nanog_classifier": "nanog_positive",
    "Sox17_classifier": "Sox17_positive",
    "NKX25_classifier": "NKX25_positive",
    "TroponinT_classifier": "TroponinT_positive",
}

rows = []

for clf_col, pos_label in CLASSIFIERS.items():
    tmp = (
        concatenated_df
        .assign(is_positive=lambda d: d[clf_col] == pos_label)
        .groupby("differentiation_day")
        .agg(
            n_cells=("is_positive", "size"),
            n_positive=("is_positive", "sum"),
        )
        .reset_index()
    )

    tmp["percent_positive"] = 100 * tmp["n_positive"] / tmp["n_cells"]
    tmp["classifier"] = clf_col

    rows.append(tmp)

summary_df = pd.concat(rows, ignore_index=True)
summary_df

,differentiation_day,n_cells,n_positive,percent_positive,classifier
0,0,134502,128907,95.840211,nanog_classifier
1,1,55680,54067,97.103089,nanog_classifier
2,2,48406,38448,79.428170,nanog_classifier
3,3,29586,7625,25.772325,nanog_classifier
4,4,66873,3665,5.480538,nanog_classifier
5,5,65437,1222,1.867445,nanog_classifier
6,6,98287,286,0.290985,nanog_classifier
7,7,107978,91,0.084276,nanog_classifier
8,8,103751,116,0.111806,nanog_classifier
9,9,86645,62,0.071556,nanog_classifier


In [28]:
summary_df.pivot(
    index="differentiation_day",
    columns="classifier",
    values="percent_positive",
)

classifier,NKX25_classifier,Sox17_classifier,TroponinT_classifier,nanog_classifier
differentiation_day,,,,
0,0.031226,0.026765,0.780658,95.840211
1,1.386494,0.829741,33.360273,97.103089
2,3.613188,0.460687,7.645746,79.428170
3,2.656662,23.879538,2.318664,25.772325
4,0.505436,24.832145,1.381724,5.480538
5,9.800266,20.272934,1.914819,1.867445
6,33.219042,0.670485,41.151933,0.290985
7,66.159773,0.053715,73.362166,0.084276
8,82.062823,0.281443,78.896589,0.111806


# Experimentation

In [88]:
concatenated_df

,label,nuclei_filtered_Morphology_area,nuclei_filtered_Morphology_centroid-0,nuclei_filtered_Morphology_centroid-1,nuclei_filtered_Morphology_well_centroid-0,nuclei_filtered_Morphology_well_centroid-1,nuclei_filtered_Morphology_bbox_area,nuclei_filtered_Morphology_bbox-0,nuclei_filtered_Morphology_bbox-1,nuclei_filtered_Morphology_bbox-2,...,nuclei_filtered_Population_mean_distance_neighbours_radius_400,nuclei_filtered_Population_n_neighbours_radius_500,nuclei_filtered_Population_mean_distance_neighbours_radius_500,well_name,ROI,nanog_classifier,Sox17_classifier,NKX25_classifier,TroponinT_classifier,differentiation_day
167,214,6109.0,87.627602,1664.032715,87.627602,9344.033203,8475.0,50.0,1613.0,125.0,...,230.168839,42.0,302.699951,B09,FOV_4,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
168,215,4718.0,92.093262,165.185242,92.093262,165.185242,6162.0,51.0,126.0,130.0,...,259.656067,24.0,309.615540,B09,FOV_1,nanog_negative,Sox17_positive,NKX25_negative,TroponinT_negative,5
169,216,4599.0,91.097412,1477.827148,91.097412,6597.827148,5920.0,51.0,1441.0,131.0,...,237.435486,40.0,292.548187,B09,FOV_3,nanog_negative,Sox17_positive,NKX25_negative,TroponinT_negative,5
170,217,8068.0,101.037308,431.152832,101.037308,2991.152832,9797.0,52.0,381.0,149.0,...,274.269928,28.0,358.872559,B09,FOV_2,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
171,219,3333.0,88.151215,1583.671509,88.151215,14383.671875,4599.0,52.0,1552.0,125.0,...,249.380447,68.0,310.089539,B09,FOV_6,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041597,73957,3611.0,2074.994385,710.316284,19354.994141,18630.316406,4900.0,2051.0,660.0,2100.0,...,215.511719,55.0,290.618256,G10,FOV_72,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6
1041603,73964,2288.0,2075.985107,260.677002,19355.984375,10500.676758,3283.0,2054.0,229.0,2103.0,...,232.565338,81.0,282.861328,G10,FOV_69,nanog_negative,Sox17_negative,NKX25_positive,TroponinT_negative,6
1041607,73970,2258.0,2083.329590,184.254211,19363.330078,10424.253906,3213.0,2056.0,150.0,2107.0,...,226.796906,73.0,285.304626,G10,FOV_69,nanog_negative,Sox17_negative,NKX25_positive,TroponinT_positive,6
1041614,73978,2402.0,2074.787598,395.848450,19354.787109,18315.847656,3034.0,2057.0,355.0,2094.0,...,243.800842,62.0,309.414673,G10,FOV_72,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_positive,6


In [64]:
na_counts = concatenated_df.isna().sum()

In [65]:
na_counts_sorted = na_counts.sort_values(ascending=False)
na_counts_sorted[na_counts_sorted > 0]

Series([], dtype: int64)

In [80]:
sum(concatenated_df["is_border_external"] == 1.0)

0

In [81]:
sum(concatenated_df["is_border_internal"] == 1.0)

0

In [ ]:
margin = 50
print(sum(concatenated_df["nuclei_filtered_Morphology_bbox-0"] < margin))
print(sum(concatenated_df["nuclei_filtered_Morphology_bbox-1"] < margin))
print(sum(concatenated_df["nuclei_filtered_Morphology_bbox-2"] > 2160 - margin))
print(sum(concatenated_df["nuclei_filtered_Morphology_bbox-3"] > 2560 -margin))

42422
35370
41367
34870


In [72]:
margin = 50

to_discard = (
    (concatenated_df["nuclei_filtered_Morphology_bbox-0"] < margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-1"] < margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-2"] > 2160 - margin)
    | (concatenated_df["nuclei_filtered_Morphology_bbox-3"] > 2560 - margin)
)
sum(to_discard)

148510

In [77]:
concatenated_df = concatenated_df.loc[~to_discard]

In [ ]:
sum(concatenated_df[~to_discard]["is_border_external"] == 1.0)

In [76]:
sum(concatenated_df[~to_discard]["is_border_internal"] == 1.0)

0

In [83]:
concatenated_df

,label,nuclei_filtered_Morphology_area,nuclei_filtered_Morphology_centroid-0,nuclei_filtered_Morphology_centroid-1,nuclei_filtered_Morphology_well_centroid-0,nuclei_filtered_Morphology_well_centroid-1,nuclei_filtered_Morphology_bbox_area,nuclei_filtered_Morphology_bbox-0,nuclei_filtered_Morphology_bbox-1,nuclei_filtered_Morphology_bbox-2,...,is_border_internal,is_border_external,well_name,ROI,segmentation_qc,nanog_classifier,Sox17_classifier,NKX25_classifier,TroponinT_classifier,differentiation_day
167,214,6109.0,87.627602,1664.032715,87.627602,9344.033203,8475.0,50.0,1613.0,125.0,...,0.0,0.0,B09,FOV_4,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
168,215,4718.0,92.093262,165.185242,92.093262,165.185242,6162.0,51.0,126.0,130.0,...,0.0,0.0,B09,FOV_1,Regular cells,nanog_negative,Sox17_positive,NKX25_negative,TroponinT_negative,5
169,216,4599.0,91.097412,1477.827148,91.097412,6597.827148,5920.0,51.0,1441.0,131.0,...,0.0,0.0,B09,FOV_3,Regular cells,nanog_negative,Sox17_positive,NKX25_negative,TroponinT_negative,5
170,217,8068.0,101.037308,431.152832,101.037308,2991.152832,9797.0,52.0,381.0,149.0,...,0.0,0.0,B09,FOV_2,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
171,219,3333.0,88.151215,1583.671509,88.151215,14383.671875,4599.0,52.0,1552.0,125.0,...,0.0,0.0,B09,FOV_6,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041597,73957,3611.0,2074.994385,710.316284,19354.994141,18630.316406,4900.0,2051.0,660.0,2100.0,...,0.0,0.0,G10,FOV_72,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_negative,6
1041603,73964,2288.0,2075.985107,260.677002,19355.984375,10500.676758,3283.0,2054.0,229.0,2103.0,...,0.0,0.0,G10,FOV_69,Regular cells,nanog_negative,Sox17_negative,NKX25_positive,TroponinT_negative,6
1041607,73970,2258.0,2083.329590,184.254211,19363.330078,10424.253906,3213.0,2056.0,150.0,2107.0,...,0.0,0.0,G10,FOV_69,Regular cells,nanog_negative,Sox17_negative,NKX25_positive,TroponinT_positive,6
1041614,73978,2402.0,2074.787598,395.848450,19354.787109,18315.847656,3034.0,2057.0,355.0,2094.0,...,0.0,0.0,G10,FOV_72,Regular cells,nanog_negative,Sox17_negative,NKX25_negative,TroponinT_positive,6


In [86]:
static_cols = concatenated_df.nunique(dropna=True) <= 5
static_columns = static_cols[static_cols].index.tolist()
static_columns

['nuclei_filtered_Morphology_euler_number',
 'is_border_internal',
 'is_border_external',
 'segmentation_qc',
 'nanog_classifier',
 'Sox17_classifier',
 'NKX25_classifier',
 'TroponinT_classifier']